In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


# Total passenger cleaning and imputation
* Total passenger is one the more important columns as it is the basis for all of our targets
such as product take rate or data usage rate per flight
* There is roughly 3% missing data and about 0.1% of flights have more passengers than seats

In [1]:
import sys
import importlib
import re
import numpy as np
import pandas as pd

sys.path.append('python')



In [2]:
pd.set_option('display.max_columns', 999)
pd.set_option('display.max_rows', 150) 



In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



In [4]:
import wifipricing.data_reader
importlib.reload(wifipricing.data_reader)
import wifipricing.sugar 
importlib.reload(wifipricing.sugar)

from wifipricing.data_reader import data_reader
from wifipricing.sugar import *



<module 'wifipricing.data_reader' from 'python/wifipricing/data_reader.py'>

<module 'wifipricing.sugar' from 'python/wifipricing/sugar.py'>

In [5]:
# Seat count, total passenger issues
seatcol=[
    'aircraft_type',
    'airline',
    'seat_count',
    'total_passengers',
    'jack_seat_count',
]

# Sample random 1e6 rows
df = data_reader(
    "data/df_rfp_dataset_raw_20181218185047.csv",
    "data/data_reference.csv",
    skiprows=np.random.randint(1, 4000000, 3000000),  #random 1e6 samples
    usecols=seatcol
)




Reading data/df_rfp_dataset_raw_20181218185047.csv with pd.read_csv()...


In [6]:
preview(df)
missing_data_report(df)



Size of dataframe: (2071127, 5)
Dataframe dimensions: (2071127, 5)
Missing data:
total_passengers:  2.94%
seat_count:  0.03%


,airline,aircraft_type,seat_count,total_passengers,jack_seat_count
0,ANA,767-300,270.0,206.875,270.0
1,ANA,767-300,270.0,206.875,270.0
2,ANA,767-300,270.0,206.875,270.0
3,ANA,767-300,270.0,206.875,270.0
4,ANA,767-300,270.0,206.875,270.0
2071122,MAS,350-900,296.0,241.375,286.0
2071123,MAS,350-900,296.0,241.375,286.0
2071124,MAS,350-900,296.0,241.375,286.0
2071125,MAS,350-900,296.0,241.375,286.0
2071126,MAS,350-900,296.0,241.375,286.0


 * On first glance, missing data isn't a huge issue with total passengers.
  However, we get some strange flights that have more passengers than seats

 * It doesn't happen too often, but we should check seat count
 * Looks like we should ignore jack_seat_count

In [7]:
df['pass_seat_ratio'] = df['total_passengers'] / df['seat_count']
df['pass_jack_ratio'] = df['total_passengers'] / df['jack_seat_count']

f"More total_passengers than seat_count: {df[df['pass_seat_ratio'] > 1].shape[0] /  df.shape[0]:.2%}"
f"More total_passengers than jack_seat_count: {df[df['pass_jack_ratio'] > 1].shape[0] /  df.shape[0]:.2%}"


'More total_passengers than seat_count: 0.10%'

'More total_passengers than jack_seat_count: 2.09%'

 ## Compare flights that have strange passenger/seat ratio to the median of that flight
 * Looks like both total_passengers and seat_count can get strange values

In [8]:
problem_ac = distinct(df[df['pass_seat_ratio'] > 1], ['aircraft_type']) 

grp_means = pd.merge(problem_ac, df, how='inner').\
    loc[:, ['aircraft_type', 'total_passengers', 'seat_count']].dropna().\
    groupby('aircraft_type').agg(lambda x: x.dropna().median()).rename(columns=lambda x: x+'_median').reset_index()

prob = distinct(df[df['pass_seat_ratio'] > 1], ['aircraft_type', 'total_passengers', 'seat_count']) 
merged = pd.merge(prob, grp_means)
merged[merged.total_passengers_median > merged.seat_count_median]


python/wifipricing/sugar.py:28: UserWarning: Missing data is dropped
  warnings.warn("Missing data is dropped")


,aircraft_type,total_passengers,seat_count,total_passengers_median,seat_count_median


 #### Same as above, but comparison with airline/aircraft type median

In [9]:
ac_al = distinct(df[df['pass_seat_ratio'] > 1], ['aircraft_type', 'airline']) 

grp_means = pd.merge(ac_al, df, how='inner').\
    loc[:, ['aircraft_type', 'airline', 'total_passengers', 'seat_count']].dropna().\
    groupby(['aircraft_type', 'airline']).median().rename(columns=lambda x: x+'_median').reset_index()

prob = distinct(df[df['pass_seat_ratio'] > 1], ['aircraft_type', 'airline', 'total_passengers', 'seat_count']) 
merged = pd.merge(prob, grp_means)
merged[merged.total_passengers_median > merged.seat_count_median]


,aircraft_type,airline,total_passengers,seat_count,total_passengers_median,seat_count_median
0,737-800,RWD,230.500,160.0,248.125,160.0
1,737-800,RWD,240.125,160.0,248.125,160.0
2,737-800,RWD,245.500,154.0,248.125,160.0
3,737-800,RWD,245.500,160.0,248.125,160.0
4,737-800,RWD,248.125,154.0,248.125,160.0
5,737-800,RWD,248.125,160.0,248.125,160.0
6,737-800,RWD,249.750,154.0,248.125,160.0
7,737-800,RWD,249.750,160.0,248.125,160.0
8,737-800,RWD,249.875,160.0,248.125,160.0
9,737-800,RWD,253.625,154.0,248.125,160.0


 ## We will change suspect total passenger/seats to median of aircraft type only

 ## Example
 ### Internal function to be used to data reader

In [10]:
def clean_passenger_seat(df):
    """
    NOTE: MODIFYING INPUT df  
    Imputing and fixing suspect total_passengers and seat_counts

    ----------------------------
    1. Missing values: both total_passengers and seat_count will get imputed by the
    median value for that airline + aircraft type 
    2. After imputation (no more missing data), replace entries where 
    total passengers > seat counts by the median of that aircraft type
    """
    req_cols = ['aircraft_type', 'airline', 'total_passengers', 'seat_count']
    assert set(req_cols).issubset(set(df.columns)), f"Input is missing one: {req_cols}"

    print('\ninput data:'); missing_data_report(df)
    print(f"Flights with more passengers than seats: {sum(df['total_passengers']/df['seat_count'] > 1) / df.shape[0]:.2%}")

    print('\nimputing total passengers and seat counts by grouped median...')
    grp_acal = df.groupby(['aircraft_type', 'airline'])
    gm_acal = grp_acal.transform(lambda x: x.dropna().median())

    df.loc[:, 'total_passengers'] = df['total_passengers'].fillna(gm_acal['total_passengers'])
    df.loc[:, 'seat_count'] = df['seat_count'].fillna(gm_acal['seat_count'])

    print('Modifying suspect passengers and seat counts by grouped median...')
    grp_ac = df.groupby(['aircraft_type'])
    gm_ac = grp_ac.transform(lambda x: x.dropna().median())
    
    ind = (df['total_passengers'] > df['seat_count'])
    df.loc[ind, 'total_passengers'] = gm_ac.loc[ind, 'total_passengers']
    df.loc[ind, 'seat_count'] = gm_ac.loc[ind, 'seat_count']

    print('\nCleaned data:'); missing_data_report(df)
    print(f"Flights with more passengers than seats: {sum(df['total_passengers']/df['seat_count'] > 1) / df.shape[0]:.2%}")
    return df



In [11]:
clean_df = clean_passenger_seat(df[['aircraft_type', 'airline', 'total_passengers', 'seat_count']].copy(deep=True))

preview(clean_df)



input data:
Dataframe dimensions: (2071127, 4)
Missing data:
total_passengers:  2.94%
seat_count:  0.03%
Flights with more passengers than seats: 0.10%

imputing total passengers and seat counts by grouped median...
Modifying suspect passengers and seat counts by grouped median...

Cleaned data:
Dataframe dimensions: (2071127, 4)
Missing data:
total_passengers:  0.1%
Flights with more passengers than seats: 0.00%
Size of dataframe: (2071127, 4)


,aircraft_type,airline,total_passengers,seat_count
0,767-300,ANA,206.875,270.0
1,767-300,ANA,206.875,270.0
2,767-300,ANA,206.875,270.0
3,767-300,ANA,206.875,270.0
4,767-300,ANA,206.875,270.0
2071122,350-900,MAS,241.375,296.0
2071123,350-900,MAS,241.375,296.0
2071124,350-900,MAS,241.375,296.0
2071125,350-900,MAS,241.375,296.0
2071126,350-900,MAS,241.375,296.0
